In [1]:
import os
from glob import glob

import dask
import dask.bag as db
import dask.dataframe as dd
from distributed import Client
from dask_jobqueue import SLURMCluster

from IPython.display import display

import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
LOCAL =True
# LOCAL = False

if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(memory='4GB', cores=1, python='/scratch/work/public/dask/bin/python', 
                               local_directory='/tmp/{}/'.format(os.environ['SLURM_JOB_USER']),
                               job_extra=['--output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])])

    cluster.submit_command = 'slurm'
    cluster.scale(100)

    display(cluster)
    client = Client(cluster)

display(client)

Client Scheduler: tcp://127.0.0.1:40419 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 4.29 GB


In [5]:
# ratings = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/ratings.csv")
# ratings = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/ratings.csv")
# mov = dd.read_csv("/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/movies.csv")
# ratings = pd.DataFrame(ratings.compute())
# mov = pd.DataFrame(mov.compute())

In [3]:
# mov
ratings = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/ratings.csv')

In [4]:
# ratings = ratings.merge(mov, how = 'right', on = "movieId")
ratings

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
27753439,283228,8542,4.5,1379882795
27753440,283228,8712,4.5,1379882751
27753441,283228,34405,4.5,1379882889
27753442,283228,44761,4.5,1354159524


In [5]:

user = pd.DataFrame(ratings.userId.unique())
train_user, testval_user = train_test_split(user, test_size=0.4)


movie = pd.DataFrame(ratings.movieId.unique())
temp1_movie, temp2_movie = train_test_split(movie, test_size = 0.7)
train_set = ratings[ratings['userId'].isin(list(train_user[0]))]
testval_set = ratings[ratings['userId'].isin(list(testval_user[0]))]
train_sending = testval_set[testval_set['movieId'].isin(list(temp1_movie[0]))]
testval_cluster =testval_set[testval_set['movieId'].isin(list(temp2_movie[0]))]

temp = pd.DataFrame(testval_set.userId.unique())
val_user, test_user = train_test_split(temp, test_size = 0.5)
val_set = testval_cluster[testval_cluster['userId'].isin(list(val_user[0]))]
test_set = testval_cluster[testval_cluster['userId'].isin(list(test_user[0]))]

In [6]:
val_set

,userId,movieId,rating,timestamp
778,5,47,4.0,1209126049
779,5,50,5.0,1209039937
781,5,296,5.0,1209040010
782,5,318,5.0,1209040104
783,5,527,4.5,1209126022
...,...,...,...,...
27753305,283226,288,2.0,1001619953
27753306,283226,344,3.0,1001619921
27753309,283226,1373,2.0,1001619967
27753311,283226,2642,2.0,1001620011


In [7]:
test_set

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
4,1,1449,4.5,1256677264
5,1,1590,2.5,1256677236
7,1,2134,4.5,1256677464
...,...,...,...,...
27752873,283221,5995,4.5,1168346995
27752874,283221,6539,3.5,1168346437
27752875,283221,30749,4.5,1168346921
27752876,283221,46965,4.0,1168358187


In [8]:
testval_set

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
27753309,283226,1373,2.0,1001619967
27753310,283226,2421,2.0,1001619992
27753311,283226,2642,2.0,1001620011
27753312,283226,2710,2.0,1001619953


In [9]:
# concatenate back to train set
final_train = train_set.append(train_sending)
# ran into concat kernel dead error
# convert it back to dask dataframe
# train_set=  dd.from_pandas(train_set, chunksize = 100000)
# train_sending = dd.from_pandas(train_sending, chunksize= 100000)

In [13]:
# final_train = dd.concat([train_set, train_sending], axis=0)

In [10]:
final_train

,userId,movieId,rating,timestamp
16,2,170,3.5,1192913581
17,2,849,3.5,1192913537
18,2,1186,3.5,1192913611
19,2,1235,3.0,1192913585
20,2,1244,3.0,1192913551
...,...,...,...,...
27753304,283226,231,2.0,1001619921
27753307,283226,420,1.0,1001619953
27753308,283226,502,1.0,1001620011
27753310,283226,2421,2.0,1001619992


In [15]:
# train_small_data = final_train
# val_small_data = val_set
# test_small_data = test_set
# train_large_data = final_train
# val_large_data = val_set
# test_large_data = test_set

In [16]:
# train_small_data.to_csv("train_small_data.csv")
# val_small_data.to_csv("val_small_data.csv")
# test_small_data.to_csv("test_small_data.csv")
# train_large_data.to_csv("train_large_data.csv")
# val_large_data.to_csv("val_large_data.csv")
# test_large_data.to_csv("test_large_data.csv")

In [11]:
final_train.to_parquet("train_large_data.parquet")

In [ ]:
val_set.to_parquet("val_large_data.parquet")

In [ ]:
test_set.to_parquet("test_large_data.parquet")